In [123]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split

from rdkit import Chem
from rdkit.Chem import AllChem

from tqdm import tqdm

In [140]:
# Define a custom Dataset class
class BiMDataset(Dataset):
    def __init__(
        self, 
        data_items, 
        unique_bonds, 
        unique_atoms, 
        bond_neighbors, 
        max_neighbors, 
        max_bond_count, 
        b_lengths_mean, 
        b_lengths_std, 
        b_angles_mean, 
        b_angles_std, 
        targets_means, 
        targets_std
    ):
        self.data_list = data_items
        self.unique_bonds = unique_bonds
        self.unique_atoms = unique_atoms
        self.bond_neighbors = bond_neighbors
        
        self.max_neighbors = max_neighbors       
        self.max_bond_count = max_bond_count
        
        self.b_lengths_mean = b_lengths_mean
        self.b_lengths_std = b_lengths_std
        
        self.b_angles_mean = b_angles_mean
        self.b_angles_std = b_angles_std
        
        self.targets_means = targets_means
        self.targets_std = targets_std
        
        self.bond_to_id = {bond: (idx+1) for idx, bond in enumerate(self.unique_bonds)}
        self.id_to_bond = {(idx + 1): bond for idx, bond in enumerate(self.unique_bonds)}

        print(f"{self.max_bond_count=}")
        
    def __len__(self):
        # Returns the number of items in the dataset
        return len(self.data_list)

    def __normalized_bond_length__(self, b_length):
        return (b_length - self.b_lengths_mean) / self.b_lengths_std

    def __normalized_bond_angle__(self, b_angle):
        return (b_angle - self.b_angles_mean) / self.b_angles_std 

    def __normalized_bond_energy__(self, b_energy):
        return (b_energy - self.targets_means) / self.targets_std 
            
    def __prepare_data__(self, item):
        smiles = item.smiles  # SMILES representation of the molecule
        mol = Chem.MolFromSmiles(smiles, sanitize=False)
        Chem.SanitizeMol(mol) 
        mol = Chem.AddHs(mol)

        bond_types = []
        bond_lengths = []
        neighbor_infos = []

        target = self.__normalized_bond_energy__(item.y[0][7]) # Internal Energy
        
        for bond in mol.GetBonds():
            atom1 = bond.GetBeginAtom()
            atom2 = bond.GetEndAtom()
            sorted_bond = tuple(sorted([atom1.GetSymbol(), atom2.GetSymbol()]))
            
            curr_bond_length = self.bond_neighbors[smiles][sorted_bond]['length']
            n_infos          = self.bond_neighbors[smiles][sorted_bond]['neighbors']
            
            bond_types.append(self.bond_to_id[sorted_bond])
            bond_lengths.append(
                self.__normalized_bond_length__(curr_bond_length)
            )

            current_bonds_neighbor_infos = [
                (
                    self.bond_to_id[n_bond], 
                    self.__normalized_bond_length__(n_bond_length), 
                    self.__normalized_bond_angle__(n_bond_angle)
                ) for n_bond, n_bond_length, n_bond_angle in n_infos
            ]
            
            # print(f"{self.max_neighbors=}")
            padded_bonds = self.max_neighbors - len(n_infos)
            # print(f"padding required={padded_bonds}")
            current_bonds_neighbor_infos += [(0, 0, 0) for _ in range(padded_bonds)]
            neighbor_infos.append(current_bonds_neighbor_infos)
           
        bond_types_tensor = torch.tensor(bond_types, dtype=torch.float32)
        bond_lengths_tensor = torch.tensor(bond_lengths, dtype=torch.float32)
        neighbor_infos_tensor = torch.tensor(neighbor_infos, dtype=torch.float32) 
        
        pad_size = self.max_bond_count - neighbor_infos_tensor.size(0)
        
        padding_tensor = torch.zeros(pad_size, neighbor_infos_tensor.size(1), neighbor_infos_tensor.size(2))
        padded_neighbor_infos_tensor = torch.cat([neighbor_infos_tensor, padding_tensor], dim=0) 

        padding_tensor = torch.zeros(pad_size)
        padded_bond_types_tensor = torch.cat([bond_types_tensor, padding_tensor.clone().detach()], dim=0)
        padded_bond_lengths_tensor = torch.cat([bond_lengths_tensor, padding_tensor.clone().detach()], dim=0)
        
        # print(f"{padded_neighbor_infos_tensor.shape=}")
        return padded_bond_types_tensor, padded_bond_lengths_tensor, padded_neighbor_infos_tensor, target
        
    def __getitem__(self, idx):
        curr_item = self.data_list[idx]
        return self.__prepare_data__(curr_item)

In [141]:
LENGTH_DIM = 1
ANGLE_DIM  = 1

class BondEmbedding(nn.Module):
    
    def __init__(self, bond_type_size, embedding_dim=16, max_neighbors=6):
        super(BondEmbedding, self).__init__()
        self.bond_type_embedding = nn.Embedding(bond_type_size, embedding_dim)
        
        # Calculate the input dimension for the fully connected layer
        combined_input_dim = (
            (embedding_dim + LENGTH_DIM)  # Current Bond Info
          + (max_neighbors * (embedding_dim + LENGTH_DIM + ANGLE_DIM)) # Neighboring Bond Info
        )

        # print(f"{combined_input_dim=}")
        
        self.fc = nn.Linear(combined_input_dim, embedding_dim)
        self.embedding_dim = embedding_dim

        
    def forward(self, bond_types, bond_lengths, neighbor_bond_types, neighbor_bond_lengths, angles):
        # Bond type embedding: [batch_size, 1] -> [batch_size, embedding_dim]
        bond_type_emb = self.bond_type_embedding(bond_types)
        # Concatenate bond embedding with its length
        bond_info = torch.cat([bond_type_emb, bond_lengths.unsqueeze(-1)], dim=-1)  # Shape: [batch_size, embedding_dim + 1]
        
        # Neighbor bond type embedding: [batch_size, max_neighbors, 1] -> [batch_size, max_neighbors, embedding_dim]
        neighbor_bond_type_emb = self.bond_type_embedding(neighbor_bond_types)
        # Concatenate neighbor bond type embedding, lengths, and angles
        neighbor_info = torch.cat([neighbor_bond_type_emb, neighbor_bond_lengths.unsqueeze(-1), angles.unsqueeze(-1)], dim=-1)  # Shape: [batch_size, max_neighbors, embedding_dim + 1 + 1]

        # Flatten neighbor_info to handle variable number of neighbors
        neighbor_info = neighbor_info.view(bond_info.size(0), bond_info.size(1), -1)  # Flatten neighbors info
        
        # print(f"{neighbor_bond_type_emb.shape=}")
        # print(f"{neighbor_info.shape=} -- {bond_info.shape=}")
        # Combine bond info and neighbor info
        
        combined_info = torch.cat([bond_info, neighbor_info], dim=-1)  # Shape: [batch_size, embedding_dim + 1 + max_neighbors * (embedding_dim + 2)]
        # print(f"{combined_info.shape=}")
        # Pass through fully connected layer for final output
        output = self.fc(combined_info)
        
        return output

In [142]:
# # Path to the saved dataset
# dataset_file = './data/QM-9-BiM-Dataset.pt'

# # Load the saved dataset
# bim_dataset = torch.load(dataset_file)

In [143]:
# bim_dataloader = DataLoader(bim_dataset, batch_size=4, shuffle=False)
# # Verify that the dataset is loaded correctly
# print(f"Dataset loaded from {dataset_file}")


In [144]:
# neighbor_infos_tensor = neighbor_infos_tensor.permute((0, 2, 3, 1))
# neighbor_infos_tensor.shape

In [145]:
# bond_types_tensor, bond_lengths_tensor, neighbor_infos_tensor, target

In [146]:
# max_bond_count = bim_dataset.max_bond_count
# max_neighbors  = bim_dataset.max_neighbors 
# unique_bond_count = len(bim_dataset.unique_bonds) + 1
# # max_bond_count, max_neighbors, unique_bond_count

In [147]:
# bond_embedding = BondEmbedding(
#     bond_type_size = unique_bond_count, 
#     embedding_dim  = 16, 
#     max_neighbors  = max_neighbors
# )
# neighbor_infos_tensor[:,:,:,0].shape, neighbor_infos_tensor[:,:,:,1].shape, neighbor_infos_tensor[:,:,:,2].shape
# output_emb = bond_embedding(
#     bond_types_tensor.to(torch.long), 
#     bond_lengths_tensor, 
#     neighbor_infos_tensor[:,:,:,0].to(torch.long), 
#     neighbor_infos_tensor[:,:,:,1], 
#     neighbor_infos_tensor[:,:,:,2]
# )

In [148]:
class BondSubnetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim=1000, output_dim=1):
        super(BondSubnetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [149]:
class BiMNetwork(nn.Module):
    def __init__(self, bond_type_size, embedding_dim, max_neighbors, max_bond_count):
        super(BiMNetwork, self).__init__()
        self.bond_embedding = BondEmbedding(bond_type_size, embedding_dim, max_neighbors)
        self.subnetworks = nn.ModuleList([
            BondSubnetwork(input_dim = self.bond_embedding.embedding_dim) for _ in range(bond_type_size)
        ])

    def forward(self, bond_types, bond_lengths, neighbor_bond_types, neighbor_bond_lengths, angles):
        bond_embeddings = self.bond_embedding(
            bond_types, 
            bond_lengths, 
            neighbor_bond_types, 
            neighbor_bond_lengths, 
            angles
        )  # [batch_size, max_bond_count, embedding_dim]

        batch_size, max_bond_count, embedding_dim = bond_embeddings.size()
        energy_per_bond = torch.zeros(batch_size, max_bond_count, device=bond_embeddings.device)
        
        for bond_type in range(len(self.subnetworks)):
            mask = (bond_types == bond_type).float().unsqueeze(-1)   # [batch_size, max_bond_count, 1]
            bond_energy = self.subnetworks[bond_type](bond_embeddings).squeeze(-1)  # [batch_size, max_bond_count]
            energy_per_bond += mask.squeeze(-1) * bond_energy       # Element-wise multiplication and aggregation
    
        molecule_energy = energy_per_bond.sum(dim=-1, keepdim=True) # [batch_size, 1]
        return molecule_energy

In [150]:
# energy_pred = bim_net(
#     bond_types_tensor.to(torch.long), 
#     bond_lengths_tensor, 
#     neighbor_infos_tensor[:,:,:,0].to(torch.long), 
#     neighbor_infos_tensor[:,:,:,1], 
#     neighbor_infos_tensor[:,:,:,2]
# )

In [151]:
# energy_pred.shape

In [152]:
def create_dataloaders(dataset, batch_size, train_split=0.9, val_split=0.1):
    train_size = int(train_split * len(dataset))
    val_size = int(val_split * len(dataset))
    test_size = len(dataset) - train_size - val_size

    train_dataset, val_dataset, test_dataset = random_split(
        dataset, [train_size, val_size, test_size]
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

In [153]:
# train_loader, val_loader, test_loader = create_dataloaders(
#     dataset = bim_dataset, 
#     batch_size = 64
# )

In [154]:
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    for bond_types, bond_lengths, neighbor_infos, targets in tqdm(train_loader, desc="Training", ncols=100):
        # Move data to the device
        bond_types = bond_types.to(torch.long).to(device)
        bond_lengths = bond_lengths.to(device)
        neighbor_bond_types = neighbor_infos[:,:,:,0].to(torch.long).to(device)

        # print(f"--  {neighbor_infos.shape=} --- ")

        neighbor_bond_lengths = neighbor_infos[:,:,:,1].to(device)
        angles = neighbor_infos[:,:,:,2].to(device)
        target = targets.to(device)

        # print(f"{angles.shape=} -- {target.shape=}")

        # Forward pass
        optimizer.zero_grad()
        predictions = model(bond_types, bond_lengths, neighbor_bond_types, neighbor_bond_lengths, angles)

        # Compute loss
        loss = criterion(predictions, target)
        total_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    return total_loss / len(train_loader)

In [155]:
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for bond_types, bond_lengths, neighbor_infos, targets in tqdm(loader, desc="Evaluating", ncols=100):
            # Move data to the device
            bond_types = bond_types.to(torch.long).to(device)
            bond_lengths = bond_lengths.to(device)
            neighbor_bond_types = neighbor_infos[:,:,:,0].to(torch.long).to(device)
            neighbor_bond_lengths = neighbor_infos[:,:,:,1].to(device)
            angles = neighbor_infos[:,:,:,2].to(device)
            target = targets.to(device)

            # Forward pass
            predictions = model(bond_types, bond_lengths, neighbor_bond_types, neighbor_bond_lengths, angles)

            # Compute loss
            loss = criterion(predictions, target)
            total_loss += loss.item()

    return total_loss / len(loader)


In [156]:
# Configuration
batch_size = 1024
learning_rate = 0.0001
num_epochs = 50000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Training on = {device}")
print(f"epochs = {num_epochs}")

Training on = cuda
epochs = 50000


In [157]:
# Load dataset and create dataloaders
dataset = torch.load('./data/QM-9-BiM-Dataset.pt')
train_loader, val_loader, test_loader = create_dataloaders(dataset, batch_size)

In [158]:
# b_lengths_mean, b_lengths_std, b_angles_mean, b_angles_std, targets_means, targets_std = get_statistics(dataset.data_list)

In [159]:
max_bond_count = dataset.max_bond_count
max_neighbors  = dataset.max_neighbors 
unique_bond_count = len(dataset.unique_bonds) + 1
print(f"{max_bond_count=} -- {max_neighbors=} -- {unique_bond_count=}")
# Initialize model, loss, and optimizer
model = BiMNetwork(
    bond_type_size = unique_bond_count, 
    embedding_dim  = 16, 
    max_neighbors  = max_neighbors, 
    max_bond_count = max_bond_count
)

max_bond_count=28 -- max_neighbors=6 -- unique_bond_count=10


In [160]:
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.95, 0.999))

# Training loop
for epoch in range(1, num_epochs+1):
    print(f"Epoch {epoch}/{num_epochs}")
    train_loss = train(model, train_loader, optimizer, criterion, device)

    if epoch % 50 == 0:
        val_loss = evaluate(model, val_loader, criterion, device)
        print(f"Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    if epoch % 200 == 0:
        # Save the model
        torch.save(model.state_dict(), f"./ckpts/model_at_epoch_{epoch}.pth")
        print(f"Model saved to ./ckpts/model_at_epoch_{epoch}.pth")

    
# Final evaluation on test data
test_loss = evaluate(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}")

# Save the model
torch.save(model.state_dict(), f"./ckpts/model_at_epoch_{num_epochs}.pth")
print(f"Model saved to ./ckpts/model_at_epoch_{num_epochs}.pth")


Epoch 1/50000


Training:  99%|██████████████████████████████████████████████████▌| 108/109 [00:17<00:00,  6.35it/s]/opt/conda/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([492])) that is different to the input size (torch.Size([492, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Training: 100%|███████████████████████████████████████████████████| 109/109 [00:17<00:00,  6.38it/s]


Epoch 2/50000


Training: 100%|███████████████████████████████████████████████████| 109/109 [00:17<00:00,  6.38it/s]


Epoch 3/50000


...

Epoch 1103/50000


Training: 100%|███████████████████████████████████████████████████| 109/109 [00:19<00:00,  5.55it/s]


Epoch 1104/50000


Training:  70%|████████████████████████████████████▎               | 76/109 [00:12<00:05,  6.24it/s]


KeyboardInterrupt: 